# 17. BERT(Bidirectional Encoder Representations from Transformers)

## 06) 한국어 BERT의 다음 문장 예측(Next Sentence Prediction)

```
!pip install transformers
```

### 1. 다음 문장 예측 모델과 토크나이저

In [1]:
import tensorflow as tf
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [4]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', 
                                                        from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


### 2. 다음 문장 예측하기

이제 TFBertForNextSentencePrediction를 통해서 다음 문장을 예측해봅시다.

모델에 입력을 넣으면, 해당 모델은 소프트맥스 함수를 지나기 전의 값인 logits을 리턴합니다. 해당 값을 소프트맥스 함수를 통과시킨 후 두 개의 값 중 더 큰 값을 모델의 예측값으로 판단하도록 더 큰 확률값을 가진 인덱스를 리턴하도록 합니다.

In [5]:
# 이어지는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


최종 예측 레이블은 0입니다. 이는 BERT가 다음 문장 예측을 학습했을 당시에 실질적으로 이어지는 두 개의 문장의 레이블은 0. 이어지지 않는 두 개의 문장의 경우에는 레이블을 1로 두고서 이진 분류로 학습을 하였기 때문입니다.

이번에는 이어지지 않는 두 개의 문장으로 테스트해봅시다.

In [6]:
# 상관없는 두 개의 문장
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]
